In [ ]:
import torch
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

from src.utils.coco_utils import make_coco_loaders
from src.models.frcnn import get_frcnn_model


def train_one_epoch(model, loader, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, targets in tqdm(loader, desc="Train FRCNN"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()
    return running_loss / len(loader)


@torch.no_grad()
def evaluate_mAP_placeholder(model, loader, device):
    # TODO: integrate proper COCO evaluator
    model.eval()
    # For now just sanity check: run forward on few batches
    for i, (images, _) in enumerate(loader):
        images = [img.to(device) for img in images]
        _ = model(images)
        if i == 10:
            break



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, val_loader = make_coco_loaders(root="/mnt/ssd2/santana-coco/data/coco", batch_size=2, num_workers=4, train_limit=5000, val_limit=1000)

num_classes = 91  # standard COCO setting (including background)
model = get_frcnn_model(num_classes=num_classes).to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)

num_epochs = 6  # short run due to limited GPU

for epoch in range(num_epochs):
    loss = train_one_epoch(model, train_loader, optimizer, device)
    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs} | loss={loss:.4f}")

    # quick sanity eval
    evaluate_mAP_placeholder(model, val_loader, device)

    torch.save(model.state_dict(), f"frcnn_epoch{epoch+1}.pth")




loading annotations into memory...
Done (t=4.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


Train FRCNN:   1%|          | 31/2500 [00:10<13:21,  3.08it/s]

In [ ]:
from ultralytics import YOLO

# choose n/s/m according to VRAM (4060: start with s)
model = YOLO("yolov8s.pt")  # pretrained on COCO

results = model.train(
    data="data/coco.yaml",
    epochs=50,
    imgsz=640,
    batch=16,      # maybe lower if OOM (e.g., 8)
    device=0,
    workers=4,
    project="runs/coco",
    name="yolov8s_640",
)

# validation metrics (mAP@[.50:.95], AP50, AP75, etc.)
model.val(data="data/coco.yaml", split="val")




New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.231 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4060, 7814MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/coco.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_6404, nbs=64, nms=False, opset=N

In [1]:
import torch
from torch.optim import AdamW
from tqdm import tqdm

from src.utils.coco_utils import make_coco_loaders
from src.models.detr_ import get_detr_model

import sys
sys.path.append("src/models/detr")  # to access detr code
from asu.matcher import HungarianMatcher
from asu.detr import SetCriterion

def train_one_epoch_detr(model, criterion, loader, optimizer, device):
    model.train()
    criterion.train()
    running_loss = 0.0

    for images, targets in tqdm(loader, desc="Train DETR"):
        images = [img.to(device) for img in images]
        # DETR expects list of dicts with 'boxes' [x0,y0,x1,y1] and 'labels'
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = model(images)  # dict with 'pred_logits', 'pred_boxes', 'aux_outputs', ...

        # Compute DETR loss
        loss_dict = criterion(outputs, targets)
        weight_dict = criterion.weight_dict

        loss = sum(
            loss_dict[k] * weight_dict[k]
            for k in loss_dict.keys()
            if k in weight_dict
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(loader)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, val_loader = make_coco_loaders(
    root="/mnt/ssd2/santana-coco/data/coco",
    batch_size=2,
    num_workers=4,
)

model = get_detr_model(num_classes=91, pretrained=True).to(device)
optimizer = AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=1e-4,
    weight_decay=1e-4,
)

# --- build criterion (from step 1) ---
num_classes = 91
matcher = HungarianMatcher(cost_class=1, cost_bbox=5, cost_giou=2)
weight_dict = {"loss_ce": 1, "loss_bbox": 5, "loss_giou": 2}
losses = ["labels", "boxes", "cardinality"]
criterion = SetCriterion(
    num_classes,
    matcher=matcher,
    weight_dict=weight_dict,
    eos_coef=0.1,
    losses=losses,
).to(device)

num_epochs = 10

for epoch in range(num_epochs):
    loss = train_one_epoch_detr(model, criterion, train_loader, optimizer, device)
    print(f"[DETR] Epoch {epoch+1} | loss={loss:.4f}")
    torch.save(model.state_dict(), f"detr_epoch{epoch+1}.pth")



ImportError: cannot import name 'detr_resnet50' from 'torchvision.models.detection' (/home/santana/MyStuff/assignments/dl-final-project/venv/lib/python3.12/site-packages/torchvision/models/detection/__init__.py)

In [ ]:
import torch
from torch.optim import AdamW
from tqdm import tqdm
from transformers import DetrImageProcessor, DetrForObjectDetection

from src.utils.coco_utils import make_coco_loaders


# ------------------------------------------------------------
# Helper: convert your targets (xyxy) → COCO-style annotations
# ------------------------------------------------------------
def build_hf_targets(targets):
    """
    Convert a batch of targets from your format:
        {
            "boxes": Tensor[num_boxes, 4] in xyxy,
            "labels": Tensor[num_boxes],
            (optionally "image_id", "area", "iscrowd")
        }
    into HuggingFace/COCO-style:
        {
            "image_id": int,
            "annotations": [
                {"bbox": [x, y, w, h], "category_id": int, "area": float, "iscrowd": 0/1},
                ...
            ]
        }
    """
    hf_targets = []

    for t in targets:
        boxes = t["boxes"]  # (N, 4), xyxy
        labels = t["labels"]

        if boxes.numel() == 0:
            annotations = []
        else:
            # xyxy → xywh
            xywh = boxes.clone()
            xywh[:, 2] = boxes[:, 2] - boxes[:, 0]  # w = x_max - x_min
            xywh[:, 3] = boxes[:, 3] - boxes[:, 1]  # h = y_max - y_min

            annotations = []
            for box, label in zip(xywh, labels):
                bbox = box.tolist()
                category_id = int(label.item() if torch.is_tensor(label) else label)
                ann = {
                    "bbox": bbox,
                    "category_id": category_id,
                    "area": float(bbox[2] * bbox[3]),  # w * h
                    "iscrowd": 0,
                }
                annotations.append(ann)

        # Keep image_id if present, otherwise 0
        if "image_id" in t:
            if torch.is_tensor(t["image_id"]):
                image_id = int(t["image_id"].item())
            else:
                image_id = int(t["image_id"])
        else:
            image_id = 0

        hf_targets.append({
            "image_id": image_id,
            "annotations": annotations,
        })

    return hf_targets


# ------------------------------------------------------------
# Training loop for HuggingFace DETR
# ------------------------------------------------------------
def train_one_epoch_detr_hf(model, processor, loader, optimizer, device):
    model.train()
    running_loss = 0.0

    for images, targets in tqdm(loader, desc="Train HF-DETR"):
        # Build HF-style targets
        hf_targets = build_hf_targets(targets)

        # Processor:
        #  - resize / pad / normalize images
        #  - convert COCO annotations into DETR labels (boxes + class labels)
        encoding = processor(
            images=images,
            annotations=hf_targets,
            return_tensors="pt",
        )

        pixel_values = encoding["pixel_values"].to(device)
        # labels is a list of dicts → move each tensor
        labels = [
            {k: v.to(device) for k, v in target.items()}
            for target in encoding["labels"]
        ]

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(loader)


# ------------------------------------------------------------
# Validation loop for HuggingFace DETR
# ------------------------------------------------------------
def validate_one_epoch_detr_hf(model, processor, loader, device):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Val HF-DETR"):
            hf_targets = build_hf_targets(targets)

            encoding = processor(
                images=images,
                annotations=hf_targets,
                return_tensors="pt",
            )

            pixel_values = encoding["pixel_values"].to(device)
            labels = [
                {k: v.to(device) for k, v in target.items()}
                for target in encoding["labels"]
            ]

            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

            running_loss += loss.item()

    return running_loss / len(loader)


# ------------------------------------------------------------
# Main: data, model, optimizer, training + validation
# ------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Your existing loaders
train_loader, val_loader = make_coco_loaders(
    root="/mnt/ssd2/santana-coco/data/coco",
    batch_size=2,
    num_workers=4,
    train_limit=500, 
    val_limit=100
)

# HuggingFace DETR (processor + model)
processor = DetrImageProcessor.from_pretrained(
    "facebook/detr-resnet-50",
    revision="no_timm",
)
model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    revision="no_timm",
).to(device)

optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

num_epochs = 10
best_val_loss = float("inf")

for epoch in range(num_epochs):
    train_loss = train_one_epoch_detr_hf(
        model=model,
        processor=processor,
        loader=train_loader,
        optimizer=optimizer,
        device=device,
    )

    val_loss = validate_one_epoch_detr_hf(
        model=model,
        processor=processor,
        loader=val_loader,
        device=device,
    )

    print(
        f"[HF-DETR] Epoch {epoch + 1} | "
        f"train_loss = {train_loss:.4f} | val_loss = {val_loss:.4f}"
    )

    # Save checkpoint every epoch
    torch.save(model.state_dict(), f"detr_hf_epoch{epoch + 1}.pth")

    # Save best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "detr_hf_best.pth")
        print(f"  -> New best model saved (val_loss = {val_loss:.4f})")



/home/santana/MyStuff/assignments/dl-final-project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
loading annotations into memory...
Done (t=4.14s)
creating index...
index created!
loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


Val HF-DETR: 100%|██████████| 1000/1000 [01:13<00:00, 13.63it/s]


[HF-DETR] Epoch 1 | train_loss = 4.8372 | val_loss = 4.5666
  -> New best model saved (val_loss = 4.5666)


Train HF-DETR:  55%|█████▌    | 1381/2500 [05:00<03:57,  4.72it/s]